In [ ]:
import pandas as pd
import os
import librosa.display
import warnings
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from google.colab import files
import os 
from google.colab import drive
drive.mount('/content/drive')
files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"mateuszkierznowski","key":"4b63d898150487f2430574b64ce90352"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c 'tensorflow-speech-recognition-challenge'
! mkdir dataset
! unzip tensorflow-speech-recognition-challenge.zip -d dataset
os.chdir('/content/dataset/')
! 7z x train.7z
os.chdir('/content/dataset/train/audio')
! unzip /content/drive/MyDrive/Pro2/silence

mkdir: cannot create directory ‘/root/.kaggle’: File exists
tensorflow-speech-recognition-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘dataset’: File exists
Archive:  tensorflow-speech-recognition-challenge.zip
replace dataset/link_to_gcp_credits_form.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/link_to_gcp_credits_form.txt  
  inflating: dataset/sample_submission.7z  
  inflating: dataset/test.7z         
  inflating: dataset/train.7z        

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1121103842 bytes (1070 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 1121103842
Headers Size = 389133
Method = Delta LZMA2:24
Solid = +
Bl

In [ ]:
df_in = pd.read_csv(r"/content/dataset/train/testing_list.txt")
 
train_audio_path = r"/content/dataset/train/audio"
 
os.chdir('/content/dataset/')

#uwaga nie ma tych silence czy tam noise
labels = ["yes", "no", "up", "down", "left", "right",
          "on", "off", "stop", "go", "zero", "one",
          "two", "three", "four", "five", "six", "seven", "eight", "nine",
          "bed", "bird", "cat", "dog", "happy", "house", "marvin", "sheila", "tree", "wow", "silence"]

In [ ]:
# all_wave = []
# all_label = []
# for label in tqdm(labels):
#     waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
#     for wav in waves:
#         samples, sample_rate = librosa.load(path=train_audio_path + '/' + label + '/' + wav)
#         features = librosa.feature.mfcc(y=samples, sr=sample_rate)
#         if (features.shape!=(20,44)): #20 i 44 sprawdzone przez print(features.shape)
#           features=np.pad(features, ((0, 20-features.shape[0]), (0, 44-features.shape[1])), "constant", constant_values=((0,0),(0,0)))
#         all_wave.append(features)
#         all_label.append(label)  
# X_num=np.array(all_wave)
# y_num=pd.get_dummies(np.array(all_label)).values.argmax(axis=1)

100%|██████████| 31/31 [1:15:43<00:00, 146.58s/it]


In [ ]:
all_wave = []
all_features = []
all_label = []
for label in tqdm(labels):
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(path=train_audio_path + '/' + label + '/' + wav)
        if samples.shape[0] < 50000:
          samples = np.pad(samples, (0, 50000 - samples.shape[0]))
        else:
          samples = samples[0:50000]
          
        features = librosa.feature.mfcc(y=samples, sr=sample_rate)
        all_wave.append(samples)
        all_features.append(features)
        all_label.append(label)  
#X_num=np.array(all_wave)
#y_num=pd.get_dummies(np.array(all_label)).values.argmax(axis=1)

100%|██████████| 31/31 [52:19<00:00, 101.27s/it]


In [ ]:
X_num=np.array(all_wave)
#X_feture=np.array(all_features)
#y_num=pd.get_dummies(np.array(all_label)).values.argmax(axis=1)

all_wave = []
all_label = []
for label in tqdm(labels):
    waves = [f for f in os.listdir(train_audio_path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(path=train_audio_path + '/' + label + '/' + wav)
        print("shape samples: {}, sr: {}".format(samples.shape, sample_rate))

In [ ]:
#os.chdir("/content/drive/MyDrive/Pro2")
#! touch X_train_noise.npy
#! touch y_train_noise.npy
#np.save(file="/content/drive/MyDrive/Pro2/X_train_noise_feat2s.npy", arr=X_feture)
np.save(file="/content/drive/MyDrive/Pro2/X_train_noise_2s.npy", arr=X_num)
#np.save(file="/content/drive/MyDrive/Pro2/y_train_noise_2s.npy", arr=y_num)
#w ten sposób masz to zapisane na google Drive od razu <3 tylko pamiętaj aby go najpierw zmountować